#Data Load

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 라이브러리

import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import VotingClassifier
import lightgbm as lgbm
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

In [ ]:
#-*- coding:utf-8 -*-

In [ ]:
!python --version
print('pandas', pd.__version__)
print('numpy', np.__version__)

Python 3.8.10
pandas 1.3.5
numpy 1.22.4


In [ ]:
# Seed 고정

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) 

In [ ]:
# 구글드라이브 활용

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/lgaimers/train.csv',encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/lgaimers/test.csv',encoding = 'utf-8')
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/lgaimers/sample_submission.csv',encoding = 'utf-8')

# Feature Engineering

> 라벨인코딩

In [ ]:
# qualitative to quantitative

qual_col = ['PRODUCT_CODE', 'LINE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i]) 
print('Done.')

Done.


In [ ]:
train_x = train.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train['Y_Class']

test_x = test.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [ ]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,4,1,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> 정규화

In [ ]:
from sklearn.preprocessing import MinMaxScaler

x_col = train.columns[train.columns.str.contains('X')].tolist()
scaler = MinMaxScaler()
  
scaler.fit(train_x[x_col])
  
train_x[x_col] = scaler.transform(train_x[x_col])
test_x[x_col] = scaler.transform(test_x[x_col])
  
train_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.256757,0.248647,0.000000,0.122283,0.890487,NaN,NaN,NaN,NaN,NaN
1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.240754,0.300866,0.407899,0.164742,0.601770,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.251422,0.133929,0.355835,0.205163,0.922566,NaN,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.199858,0.202110,0.704129,0.003057,0.559181,NaN,NaN,NaN,NaN,NaN
4,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.233997,0.275703,0.515978,0.088315,0.846239,NaN,NaN,NaN,NaN,NaN


> 결측값 'train_x' 평균으로 채우기

In [ ]:
train_x = train_x.fillna(train_x.mean())
test_x = test_x.fillna(train_x.mean())

> 전체 결측값(Null)인 X변수 Drop하기

In [ ]:
train_x = train_x.dropna(axis =1)
test_x = test_x.dropna(axis=1)

# Modeling

In [ ]:
train_x.shape, train_y.shape, test_x.shape

((598, 2795), (598,), (310, 2795))

In [ ]:
#lightGBM, ExtraTreesClassifier, RandomForestClassifier 선택

model = lgbm.LGBMClassifier()
et_cls = ExtraTreesClassifier(n_estimators=500, min_samples_leaf=5, min_samples_split=7, max_features=2795)
rf_cls = RandomForestClassifier(n_estimators=500, min_samples_leaf=5, min_samples_split=7, max_features=2795)
lg_cls = model.fit(train_x, train_y)

# 모델 voting

voting = VotingClassifier(
    estimators=[
        ('et', et_cls),
        ('rf', rf_cls),
        ('lg', lg_cls)
    ]
)
voting.fit(train_x, train_y)

VotingClassifier(estimators=[('et',
                              ExtraTreesClassifier(max_features=2795,
                                                   min_samples_leaf=5,
                                                   min_samples_split=7,
                                                   n_estimators=500)),
                             ('rf',
                              RandomForestClassifier(max_features=2795,
                                                     min_samples_leaf=5,
                                                     min_samples_split=7,
                                                     n_estimators=500)),
                             ('lg', LGBMClassifier())])

# Predict

In [ ]:
pred = voting.predict(test_x)
sub['Y_Class'] = pred

sub.to_csv('./answer.csv', index=False)